In [28]:
import sys, os

# force PYTHONPATH to look into the project directory for modules
rootdir = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.insert(0, rootdir)


# Pooling-imputation performance from real bead chip data

Performs pooling simulation and imputation on data from the chromosome 20 of samples in the 1000 Genomes Project.
The 1000 first markers have been selected and all unrelated individuals (2504 samples). 
These samples are randomly subdivided into a reference panel REF and a study population IMP to be pooled and imputed.
Imputation is done with Beagle 4.1 and the default parameters.


In [29]:
print('Check number of samples')
%sx bcftools query -l ALL.chr20.snps.gt.vcf.gz | wc -l

Check number of samples


['2504']

In [30]:
print('Check number of intersected markers')
%sx bcftools view -H ALL.chr20.snps.gt.vcf.gz | wc -l

Check number of intersected markers


['1000']

In [31]:
print('Shuffle split into file for REF and IMP populations')
from genotypooler.poolSNPs import poolvcf
from genotypooler.poolSNPs.pooler import Design

ds = Design()
dm = ds.matrix

sfsp = poolvcf.ShuffleSplitVCF(dm, 'ALL.chr20.snps.gt.vcf.gz', stu_size=0.1, wd=os.getcwd())
sfsp.split_file('chr20.snps.gt.vcf.gz')

Shuffle split into file for REF and IMP populations
/home/camcl609/genotypooler/examples/REF.chr20.snps.gt.vcf.gz:
 File created? -> True
/home/camcl609/genotypooler/examples/REF.chr20.snps.gt.vcf.gz:
 File indexed? -> True
/home/camcl609/genotypooler/examples/IMP.chr20.snps.gt.vcf.gz:
 File created? -> True
/home/camcl609/genotypooler/examples/IMP.chr20.snps.gt.vcf.gz:
 File indexed? -> True


In [32]:
print('Apply pooling on the study population')
# get processor characteristics on Linux-based OS
%sx cat /proc/cpuinfo  | grep 'name'| uniq
# model name	: Intel(R) Core(TM) i7-7600U CPU @ 2.80GHz

Apply pooling on the study population


['model name\t: Intel(R) Core(TM) i7-10850H CPU @ 2.70GHz']

In [33]:
#pooling simulation
%sx python3 -u pooling-ex.py IMP.chr20.snps.gt.vcf.gz IMP.chr20.pooled.snps.gl.vcf.gz GP
# Time for pooling 1000 variants = 6.537596869000481 sec

['',
 '*******************************************************************************',
 'Input file = IMP.chr20.snps.gt.vcf.gz',
 'Output file = IMP.chr20.pooled.snps.gl.vcf.gz',
 '*******************************************************************************',
 '',
 'Pooling data in /home/camcl609/genotypooler/examples/IMP.chr20.snps.gt.vcf.gz.....',
 '1 variants processed in 000.01 sec..............................................',
 '',
 'Writing metadata in /home/camcl609/genotypooler/examples/IMP.chr20.pooled.snps.gl.vcf',
 '',
 'Writing data in /home/camcl609/genotypooler/examples/IMP.chr20.pooled.snps.gl.vcf',
 'Writing data in /home/camcl609/genotypooler/examples/IMP.chr20.pooled.snps.gl.vcf: Done',
 '/home/camcl609/genotypooler/examples/IMP.chr20.pooled.snps.gl.vcf.gz:',
 ' File created? -> True',
 '/home/camcl609/genotypooler/examples/IMP.chr20.pooled.snps.gl.vcf.gz:',
 ' File indexed? -> True',
 'Time for pooling 1000 variants = 6.247895957001674 sec',
 '',
 'Time elapsed

In [34]:
print('Impute missing genotypes in the pooled file')
%sx cp ../bin/beagle_pipeline.sh ./
%sx bash beagle_pipeline.sh

Impute missing genotypes in the pooled file


['Contigs in the reference file',
 '.................................................................................',
 'Chromosome  20    Startpos = 60343    Endpos = 99420',
 '',
 '',
 'Check FORMAT field in files for imputation',
 '.................................................................................',
 'FORMAT in reference panel:  GT',
 'FORMAT in target:  GL',
 '',
 '',
 'Check number of samples and number of markers in files for imputation',
 '.................................................................................',
 'reference:',
 '2264',
 '',
 'target:',
 '240',
 '',
 '',
 'Phase reference and target with BEAGLE',
 '.................................................................................',
 'Beagle .jar file used at: ../bin/beagle.11Mar19.69c.jar',
 '',
 'FORMAT in the phased ref file: GT',
 'beagle.11Mar19.69c.jar (version 4.1)',
 'Copyright (C) 2014-2015 Brian L. Browning',
 'Enter "java -jar beagle.11Mar19.69c.jar" for a summary of command lin

In [35]:
# Verify files created at the different phasing and imputation steps
assert os.path.exists('IMP.chr20.pooled.unphased.vcf.gz')
assert os.path.exists('IMP.chr20.pooled.phased.vcf.gz')
assert os.path.exists('IMP.chr20.pooled.phased.dedup.vcf.gz')
assert os.path.exists('IMP.chr20.pooled.imputed.vcf.gz')

### Compute results with customized metrics
Show classification task-based metrics and classical genotype imputation metrics

In [36]:
%sx python3 -u ../poolSNPs/imputation_quality.py ./ IMP.chr20.snps.gt.vcf.gz IMP.chr20.pooled.imputed.vcf.gz ../bin/gt_to_log_gl.sh 'id'  

['/home/camcl609',
 '/home/camcl609/genotypooler/poolSNPs/metrics/quality.py:329: RuntimeWarning: divide by zero encountered in myprodlog_numba',
 '  result = myprodlog_numba(dftrue.values, dfpred.values)',
 '/home/camcl609/genotypooler/poolSNPs/metrics/quality.py:329: RuntimeWarning: divide by zero encountered in myprodlog_numba',
 '  result = myprodlog_numba(dftrue.values, dfpred.values)',
 '/home/camcl609/.local/lib/python3.8/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.',
 '  warnings.warn(PearsonRConstantInputWarning())',
 '/home/camcl609/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.',
 '  _warn_prf(average, modifier, msg_start, len(result))',
 '/home/camcl609/.local/lib/python3.8/site-packages/s

In [37]:
# Verify files created at the different phasing and imputation steps
assert os.path.exists('imputation_quality_gtgl.pdf')


In [38]:
# # if bcftools is configured for python 2.7 usage 
# print('Plotting results with bcftools stats')
# %sx deactivate
# # bcftools stats needs python 2.7
# %sx bcftools stats --af-bins 0.01,0.02,0.04,0.08,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.98 --collapse snps -S study.population IMP.chr20.pooled.imputed.vcf.gz IMP.chr20.snps.gt.vcf.gz > filestats.vchk
# %sx plot-vcfstats -p bcftoolstats -s filestats.vchk
